In [1]:
#we start by uploading to our buckets the joined dfs both .parquet & .csv, in order the comparison to make sense

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1260,application_1732639283265_1223,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType
from pyspark.sql.functions import year, to_date
from pyspark.sql.functions import substring
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.functions import count, when, length
import time

from pyspark.sql.functions import count, when, col
from pyspark.sql import Window
from pyspark.sql.functions import col, row_number, round

spark = SparkSession \
    .builder \
    .appName("DF query 2 execution") \
    .getOrCreate()

# Access configuration
conf = spark.sparkContext.getConf()

# Print relevant executor settings
print("Executor Instances:", conf.get("spark.executor.instances"))
print("Executor Memory:", conf.get("spark.executor.memory"))
print("Executor Cores:", conf.get("spark.executor.cores"))



Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3010,application_1732639283265_2968,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Executor Instances: None
Executor Memory: 4743M
Executor Cores: 2

In [2]:
victim19 = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv",
    header=True
)

victim20 = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv",
    header=True
)

victim_df19 = victim19.select("AREA ", "Status", "DATE OCC")

victim_df20 = victim20.select("AREA", "Status", "DATE OCC")
victim_df20 = victim_df20.withColumnRenamed("AREA", "AREA ")


# Combine the two datasets
victim_df = victim_df19.union(victim_df20)

joined_data = victim_df

#joined_data.show(10)

police_station = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/LA_Police_Stations.csv",
    header=True
)

#police_station.show(3)
joined_data = joined_data.withColumn(
    "AREA ",
    when(col("AREA ") == "01", 1)
    .when(col("AREA ") == "02", 2)
    .when(col("AREA ") == "03", 3)
    .when(col("AREA ") == "04", 4)
    .when(col("AREA ") == "05", 5)
    .when(col("AREA ") == "06", 6)
    .when(col("AREA ") == "07", 7)
    .when(col("AREA ") == "08", 8)
    .when(col("AREA ") == "09", 9)
    .when(col("AREA ") == "10", 10)
    .when(col("AREA ") == "11", 11)
    .when(col("AREA ") == "12", 12)
    .when(col("AREA ") == "13", 13)
    .when(col("AREA ") == "14", 14)
    .when(col("AREA ") == "15", 15)
    .when(col("AREA ") == "16", 16)
    .when(col("AREA ") == "17", 17)
    .when(col("AREA ") == "18", 18)
    .when(col("AREA ") == "19", 19)
    .when(col("AREA ") == "20", 20)
    .when(col("AREA ") == "21", 21)
)
police_station = police_station.withColumn(
    "PREC",
    when(col("PREC") == "1", 1)
    .when(col("PREC") == "2", 2)
    .when(col("PREC") == "3", 3)
    .when(col("PREC") == "4", 4)
    .when(col("PREC") == "5", 5)
    .when(col("PREC") == "6", 6)
    .when(col("PREC") == "7", 7)
    .when(col("PREC") == "8", 8)
    .when(col("PREC") == "9", 9)
    .when(col("PREC") == "10", 10)
    .when(col("PREC") == "11", 11)
    .when(col("PREC") == "12", 12)
    .when(col("PREC") == "13", 13)
    .when(col("PREC") == "14", 14)
    .when(col("PREC") == "15", 15)
    .when(col("PREC") == "16", 16)
    .when(col("PREC") == "17", 17)
    .when(col("PREC") == "18", 18)
    .when(col("PREC") == "19", 19)
    .when(col("PREC") == "20", 20)
    .when(col("PREC") == "21", 21)
)
#start_time = time.time()

# df1.join(df2, df1['id'] == df2['id'], 'inner')
joined_data = joined_data.withColumn(
    "YEAR", 
    when(col("DATE OCC").isNotNull() & (length("DATE OCC") >= 7), substring("DATE OCC", 7, 4)).otherwise(None)
)
joined_data.show(3)



joined_data = joined_data.join(police_station, joined_data['AREA '] == police_station['PREC'], 'inner')

joined_data = joined_data.select("AREA ", "PREC", "DIVISION", "YEAR", "Status") \
                         .orderBy("AREA ")

joined_data=joined_data.select("AREA ","DIVISION", "YEAR", "Status")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------+--------------------+----+
|AREA |Status|            DATE OCC|YEAR|
+-----+------+--------------------+----+
|   13|    AA|02/20/2010 12:00:...|2010|
|   14|    IC|09/12/2010 12:00:...|2010|
|   13|    IC|08/09/2010 12:00:...|2010|
+-----+------+--------------------+----+
only showing top 3 rows

In [3]:
#we upload both .parquet and .csv with the all data that we need

parquet_s3_path = "s3://groups-bucket-dblab-905418150721/group17/query2_data.parquet"
joined_data.write.mode("overwrite").parquet(parquet_s3_path)

csv_s3_path = "s3://groups-bucket-dblab-905418150721/group17/query2_data.csv"
joined_data.write.mode("overwrite").option("header", "true").csv(csv_s3_path)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…